<a href="https://colab.research.google.com/github/Initiator-Z/Project_Zero/blob/master/42028A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 42028 Deep Learning and Convolutional Networks Assignment 1

# **Dataset Import**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# import packges and load dataset

import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
%matplotlib inline
import cv2
import seaborn as sns

def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    labels_path = os.path.join(path,'%s-labels-idx1-ubyte.gz'% kind)
    images_path = os.path.join(path,'%s-images-idx3-ubyte.gz'% kind)

    with gzip.open(labels_path, 'rb') as lbpath:labels = np.frombuffer(lbpath.read(), dtype=np.uint8,offset=8)

    with gzip.open(images_path, 'rb') as imgpath:images = np.frombuffer(imgpath.read(), dtype=np.uint8,offset=16).reshape(len(labels), 784)

    return images, labels

X_train, y_train = load_mnist('/content/gdrive/My Drive/42028-DL-CNN-2020/Assignment1/Data', kind='train')
X_test, y_test = load_mnist('/content/gdrive/My Drive/42028-DL-CNN-2020/Assignment1/Data', kind='t10k')

labelNames = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

In [0]:
X_train=X_train.reshape(-1,28,28)
X_test=X_test.reshape(-1,28,28)

print("Train dataset after reshaping:{}".format(np.shape(X_train)))
print("Test dataset after reshaping :{}".format(np.shape(X_test)))

Train dataset after reshaping:(60000, 28, 28)
Test dataset after reshaping :(10000, 28, 28)


#**Feature Extraction**

In [0]:
# LBP feature extraction

class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius
        
    def LBPfeatures(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method='uniform')
        (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, self.numPoints+3),range=(0, self.numPoints+2))
        hist = hist.astype('float')
        hist /= (hist.sum()+eps)
        return hist

desc = LocalBinaryPatterns(24,8)
lbp_data_train = []
lbp_label_train = []

for img_index in range(len(X_train)):
    image = (X_train[img_index])
    hist = desc.LBPfeatures(image)
    lbp_label_train.append(y_train[img_index])
    lbp_data_train.append(hist)

In [0]:
# HOG feature extraction

hog_data_train = []
hog_label_train = []

for img_index in range(len(X_train)):
  image = (X_train[img_index])
  H = feature.hog(image, orientations=9, pixels_per_cell=(4, 4),
                  cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1-sqrt")
 
  hog_data_train.append(H)
  hog_label_train.append(y_train[img_index])

In [0]:
# Feature vector extraction

raw_data_train = []
raw_label_train = []

for img_index in range(len(X_train)):
    image = (X_train[img_index])
    raw = image.flatten()
    
    raw_label_train.append(y_train[img_index])
    raw_data_train.append(raw)

np.shape(raw_data_train)

(60000, 784)

# K Nearest Neighbour


In [0]:
# train and evaluate KNN classifier on LBP feature

knn = KNeighborsClassifier(n_neighbors = 6)
knn.fit(lbp_data_train, lbp_label_train)

predictions=[]
predict_label=[]
for im_index in range(len(X_test)):
  imag = X_test[im_index]
  histo = desc.LBPfeatures(imag)
  prediction = knn.predict(histo.reshape(1, -1))
  predictions.append(prediction)
  predict_label.append(y_test[im_index])

accuracy_knn_lbp = metrics.accuracy_score(y_test, predictions)
print("Test set accuracy of KNN on LBP:",accuracy_knn_lbp)

In [0]:
# confusion matrix for KNN on LBP

cm  = metrics.confusion_matrix(y_test, predictions)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(accuracy_knn_lbp)
plt.title(all_sample_title, size = 15);

In [0]:
# train and evaluate KNN classifier on HOG feature

knn = KNeighborsClassifier(n_neighbors = 6)
knn.fit(hog_data_train, hog_label_train)

predict_test = []
label_test = []
data_test=[]
for img_ind in range(len(X_test)):
  img=X_test[img_ind]
  H = feature.hog(img, orientations=9, pixels_per_cell=(4, 4),cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1-sqrt")
  pred = knn.predict(H.reshape(1, -1))[0]
  predict_test.append(pred)
  data_test.append(H)
  label_test.append(y_test[img_ind])
  
accuracy_knn_hog = metrics.accuracy_score(y_test, predict_test)
print("Accuracy on test dataset:",accuracy_knn_hog)

In [0]:
# confusion matrix for KNN on HOG
cm  = metrics.confusion_matrix(y_test, predict_test)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(accuracy_knn_hog)
plt.title(all_sample_title, size = 15);

In [0]:
# train and evaluate KNN on raw input

knn = KNeighborsClassifier(n_neighbors = 6)
knn.fit(raw_data_train, raw_label_train)

predicts = []
label_test = []
for img_index in range(len(X_test)):
    image = (X_test[img_index])
    raw = image.flatten()
    pred = knn.predict(raw.reshape(1,-1))
    predicts.append(pred)
    label_test.append(y_test[img_ind])

accuracy_knn_raw = metrics.accuracy_score(y_test, predicts)
print("Test set Accuracy of KNN: ",accuracy_knn_raw)

In [0]:
# confusion matrix for KNN on raw input
cm  = metrics.confusion_matrix(y_test, predicts)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(accuracy_knn_raw)
plt.title(all_sample_title, size = 15);

In [0]:
# Examples of knn prediction on test set with raw inputs
images = []
 
for i in np.random.choice(np.arange(0, len(y_test)), size=(16,)):
  image = (X_test[i]).flatten()
  prediction = knn.predict(image.reshape(1, -1))
  label = labelNames[prediction[0]]
  orig_label=labelNames[y_test[i]]
  image = X_test[i]
  image = cv2.merge([image] * 3)
  image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
  cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.50, color, 2)
  images.append(image)

fig = plt.figure()
ax1 = fig.add_subplot(2,2,1)
ax1.imshow(images[1])
print(orig_label[:])
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(images[2])
ax3 = fig.add_subplot(2,2,3)
ax3.imshow(images[3])
ax4 = fig.add_subplot(2,2,4)
ax4.imshow(images[4])

# Support Vector Machine

In [0]:
# train and evaluate SVM classifier on LBP feature

svm = SVC(kernel='rbf',C=1000, random_state=42)
svm.fit(lbp_data_train, lbp_label_train)

predictions=[]
predict_label=[]
for im_index in range(len(X_test)):
  imag = X_test[im_index]
  histo = desc.LBPfeatures(imag)
  prediction = svm.predict(histo.reshape(1, -1))
  predictions.append(prediction)
  predict_label.append(y_test[im_index])

accuracy_svm_lbp = metrics.accuracy_score(y_test, predictions)
print("Test set accuracy of KNN on LBP:",accuracy_svm_lbp)

In [0]:
# confusion matrix for SVM on LBP

cm  = metrics.confusion_matrix(y_test, predictions)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(accuracy_svm_lbp)
plt.title(all_sample_title, size = 15);

In [0]:
# train and evaluate SVM classifier on HOG feature

svm = SVC(kernel='rbf',C=1000, random_state=42)
svm.fit(hog_data_train, hog_label_train)

predict_test = []
label_test = []
data_test=[]
for img_index in range(len(X_test)):
  img=X_test[img_index]
  H = feature.hog(img, orientations=9, pixels_per_cell=(4, 4),cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1-sqrt")
  pred = svm.predict(H.reshape(1, -1))[0]
  predict_test.append(pred)
  data_test.append(H)
  label_test.append(y_test[img_index])

accuracy_svm_hog = metrics.accuracy_score(y_test, predict_test)
print("Accuracy on test dataset:",accuracy_svm_hog)

In [0]:
# confusion matrix for SVM on HOG
cm  = metrics.confusion_matrix(y_test, predict_test)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(accuracy_svm_hog)
plt.title(all_sample_title, size = 15);

In [0]:
# Examples of svm prediction on HOG feature

images = []
orig_labels=[]
for i in np.random.choice(np.arange(0, len(y_test)), size=(16,)):
  test_img = (X_test[i])
  H1 = feature.hog(test_img, orientations=9, pixels_per_cell=(4, 4),cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2-Hys")
  pred = svm.predict(H1.reshape(1, -1))[0]
  label = labelNames[pred]
  orig_labels.append(labelNames[y_test[i]])
  color = (0, 255, 0)
  test_img = cv2.merge([test_img] * 3)
  test_img = cv2.resize(test_img, (96, 96), interpolation=cv2.INTER_LINEAR)
  cv2.putText(test_img, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.50, color, 2)
  images.append(test_img)

orig_labels[1]
fig = plt.figure()
ax1 = fig.add_subplot(2,2,1)
ax1.imshow(images[1])
print(orig_labels[1])
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(images[2])
print(orig_labels[2])
ax3 = fig.add_subplot(2,2,3)
ax3.imshow(images[3])
print(orig_labels[3])
ax4 = fig.add_subplot(2,2,4)
ax4.imshow(images[4])
print(orig_labels[4])

In [0]:
# train and evaluate SVM on raw input

svm = SVC(kernel='rbf',C=1000, random_state=42)
svm.fit(raw_data_train, raw_label_train)

predicts = []
label_test = []
for img_index in range(len(X_test)):
    image = (X_test[img_index])
    raw = image.flatten()
    pred = svm.predict(raw.reshape(1,-1))
    predicts.append(pred)
    label_test.append(y_test[img_index])

accuracy_svm_raw = metrics.accuracy_score(y_test, predicts)
print("Test set Accuracy of KNN: ",accuracy_svm_raw)

In [0]:
# confusion matrix for SVM on raw input

cm  = metrics.confusion_matrix(y_test, predicts)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(accuracy_svm_raw)
plt.title(all_sample_title, size = 15);

# Artificial Neural Network

In [0]:
# train and evaluate neural network on LBP feature

import tensorflow as tf
from tensorflow import keras

lbp_data_train = np.array(lbp_data_train)
lbp_label_train = np.array(lbp_label_train)

lbp_image_valid = lbp_data_train[:5000]
lbp_labels_valid = lbp_label_train[:5000]
lbp_image_train = lbp_data_train[5000:]
lbp_labels_train = lbp_label_train[5000:]

model = tf.keras.models.Sequential([tf.keras.layers.Dense(256, activation=tf.nn.relu), tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer='nadam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

H = model.fit(lbp_image_train, lbp_labels_train, epochs=10, validation_data=(lbp_image_valid, lbp_labels_valid))

lbp_data_test = []
lbp_label_test = []

for img_index in range(len(X_test)):
    image = (X_test[img_index])
    hist = desc.LBPfeatures(image)
    lbp_label_test.append(y_test[img_index])
    lbp_data_test.append(hist)

acc1 = model.evaluate(np.array(lbp_data_test),np.array(lbp_label_test))

Epoch 1/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.8828 - accuracy: 0.3306 - val_loss: 1.6553 - val_accuracy: 0.4308
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.5945 - accuracy: 0.4389 - val_loss: 1.5539 - val_accuracy: 0.4462
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.5314 - accuracy: 0.4596 - val_loss: 1.5052 - val_accuracy: 0.4726
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.5051 - accuracy: 0.4714 - val_loss: 1.4931 - val_accuracy: 0.4720
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.4874 - accuracy: 0.4779 - val_loss: 1.4679 - val_accuracy: 0.4804
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.4723 - accuracy: 0.4843 - val_loss: 1.4530 - val_accuracy: 0.4862
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.4586 - accuracy: 0.4907 - val_loss: 1.4475 - val_accuracy:

In [0]:
# Confusion matrix for nn on LBP

y_pred = model.predict(np.array(lbp_data_test))
pred = np.argmax(y_pred, axis=1)
cm  = metrics.confusion_matrix(np.array(y_test), pred)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Test set Accuracy:{}'.format(acc1[1:])
plt.title(all_sample_title, size = 15);

In [0]:
# train and evaluate neural network on HOG feature

hog_data_train = np.array(hog_data_train)
hog_label_train = np.array(hog_label_train)

hog_image_valid = hog_data_train[:5000]
hog_labels_valid = hog_label_train[:5000]
hog_image_train = hog_data_train[5000:]
hog_labels_train = hog_label_train[5000:]

model2 = tf.keras.models.Sequential([tf.keras.layers.Dense(256, activation=tf.nn.relu), tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model2.compile(optimizer='nadam',loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

H = model2.fit(hog_image_train, hog_labels_train, epochs=10,validation_data=(hog_image_valid, hog_labels_valid))

hog_data_test = []
hog_label_test = []
for img_index in range(len(X_test)):
  image = (X_test[img_index])
  H = feature.hog(image, orientations=9, pixels_per_cell=(4, 4),
                  cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1-sqrt")
  hog_data_test.append(H)
  hog_label_test.append(y_test[img_index])

acc2 = model2.evaluate(np.array(hog_data_test),np.array(hog_label_test))


Epoch 1/10
1719/1719 [==============================] - 19s 11ms/step - loss: 0.1126 - accuracy: 0.9666 - val_loss: 0.0773 - val_accuracy: 0.9748
Epoch 2/10
1719/1719 [==============================] - 19s 11ms/step - loss: 0.0490 - accuracy: 0.9843 - val_loss: 0.0513 - val_accuracy: 0.9822
Epoch 3/10
1719/1719 [==============================] - 19s 11ms/step - loss: 0.0302 - accuracy: 0.9899 - val_loss: 0.0481 - val_accuracy: 0.9842
Epoch 4/10
1719/1719 [==============================] - 20s 11ms/step - loss: 0.0209 - accuracy: 0.9933 - val_loss: 0.0434 - val_accuracy: 0.9858
Epoch 5/10
1719/1719 [==============================] - 20s 11ms/step - loss: 0.0137 - accuracy: 0.9956 - val_loss: 0.0422 - val_accuracy: 0.9880
Epoch 6/10
1719/1719 [==============================] - 19s 11ms/step - loss: 0.0119 - accuracy: 0.9960 - val_loss: 0.0412 - val_accuracy: 0.9882
Epoch 7/10
1719/1719 [==============================] - 19s 11ms/step - loss: 0.0078 - accuracy: 0.9975 - val_loss: 0.0559 -

In [0]:
# Confusion matrix for nn on HOG

y_pred = model2.predict(np.array(hog_data_test))
pred = np.argmax(y_pred, axis=1)
cm  = metrics.confusion_matrix(np.array(y_test), pred)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Test set Accuracy: {0}'.format(acc2[1:])
plt.title(all_sample_title, size = 15);

In [0]:
# train and evaluate neural network on raw input

import tensorflow as tf
from tensorflow import keras

raw_data_train = np.array(raw_data_train)
raw_label_train = np.array((raw_label_train))

raw_image_valid = np.array(raw_data_train[:5000])/255.0
raw_labels_valid = raw_label_train[:5000]
raw_image_train = np.array(raw_data_train[5000:])/255.0
raw_labels_train = raw_label_train[5000:]

model3 = tf.keras.models.Sequential([tf.keras.layers.Dense(256, activation=tf.nn.relu), tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model3.compile(optimizer='nadam',loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

H = model3.fit(raw_image_train, raw_labels_train, epochs=10,validation_data=(raw_image_valid, raw_labels_valid))

X_test = X_test/255.0

raw_data_test = []
raw_label_test = []
for img_index in range(len(X_test)):
    image = (X_test[img_index])
    raw = image.flatten()
    raw_data_test.append(raw)
    raw_label_test.append(y_test[img_index])

acc3 = model3.evaluate(np.array(raw_data_test),np.array(raw_label_test))

Epoch 1/10
1719/1719 [==============================] - 13s 8ms/step - loss: 0.2074 - accuracy: 0.9397 - val_loss: 0.0964 - val_accuracy: 0.9710
Epoch 2/10
1719/1719 [==============================] - 13s 8ms/step - loss: 0.0823 - accuracy: 0.9751 - val_loss: 0.0785 - val_accuracy: 0.9780
Epoch 3/10
1719/1719 [==============================] - 13s 8ms/step - loss: 0.0517 - accuracy: 0.9833 - val_loss: 0.0719 - val_accuracy: 0.9770
Epoch 4/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.0361 - accuracy: 0.9880 - val_loss: 0.0720 - val_accuracy: 0.9776
Epoch 5/10
1719/1719 [==============================] - 13s 8ms/step - loss: 0.0268 - accuracy: 0.9907 - val_loss: 0.0739 - val_accuracy: 0.9778
Epoch 6/10
1719/1719 [==============================] - 13s 8ms/step - loss: 0.0204 - accuracy: 0.9936 - val_loss: 0.0772 - val_accuracy: 0.9786
Epoch 7/10
1719/1719 [==============================] - 13s 8ms/step - loss: 0.0170 - accuracy: 0.9945 - val_loss: 0.0677 - val_ac

In [0]:
# Confusion matrix for nn on raw inputs

y_pred = model3.predict(np.array(raw_data_test))
pred = np.argmax(y_pred, axis=1)
cm  = metrics.confusion_matrix(np.array(y_test), pred)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Test set Accuracy: {0}'.format(acc3[1:])
plt.title(all_sample_title, size = 15);